In [10]:
import numpy as np
import pandas as pd
import plotly.express as px

import json

In [11]:
df_train = pd.read_parquet("datasets/binned_train.parquet")
df_test = pd.read_parquet("datasets/binned_test.parquet")

In [12]:
df_train[["Opening", "ECO"]].head(10)

,Opening,ECO
0,Vienna Game: Stanley Variation,C26
1,French Defense: Schlechter Variation,C00
2,Sicilian Defense: Old Sicilian,B30
3,Caro-Kann Defense: MarÃ³czy Variation,B12
4,Duras Gambit,B00
5,Russian Game,C42
6,King's Indian Defense: Smyslov Variation,E61
7,"Caro-Kann Defense: Advance Variation, Botvinni...",B12
8,Queen's Pawn Game: Mason Variation,D00
9,Owen Defense,B00


## Mean Encoding

In [13]:
opening_features = ["Opening", "ECO"]
for feature in opening_features:
    
    # Распространенные дебюты
    common_openings = (
        df_train[feature]
        .value_counts()
        .where(lambda x: x >= 100).dropna()
        .index
    )
    
    # Считаем средние
    means_dict = (
        df_train
        .groupby(feature)
        .agg({"Elo": "mean"})
        .loc[common_openings]
        .squeeze().to_dict()
    )
    
    # Среднее для редких
    mean_if_rare = df_train["Elo"].where(
        ~df_train[feature].isin(common_openings)
    ).mean()
    
    # Применяем
    df_train[feature] = (
        df_train[feature]
        .map(means_dict)
        .fillna(mean_if_rare)
    )
    
    df_test[feature] = (
        df_test[feature]
        .map(means_dict)
        .fillna(mean_if_rare)
    )

In [14]:
df_train.to_parquet("datasets/final_train.parquet")
df_test.to_parquet("datasets/final_test.parquet")

## Корзина

In [15]:
# df_train["OpeningType"] = df_train["Opening"].str.split(":", expand=True)[0]
# df_train["ECOType"] = df_train["ECO"].str[:2]